## Getting Mail Ids from GMail using Pagination

In [1]:
import boto3

In [2]:
sm_client = boto3.client(
    'secretsmanager',
    region_name='us-east-1'
)

In [3]:
secret_token = sm_client.get_secret_value(SecretId='gmail_token')['SecretBinary']

* `secret_token` contains the content of token.pickle generated when we have authenticated with our gmail account to read emails.

In [4]:
import pickle

In [5]:
creds = pickle.loads(secret_token) # converting the content of token.pickle to credentials to GMail API

In [6]:
from googleapiclient.discovery import build

In [7]:
service = build('gmail', 'v1', credentials=creds)

In [8]:
users = service.users()

In [9]:
help(users.messages().list)

Help on method method in module googleapiclient.discovery:

method(**kwargs) method of googleapiclient.discovery.Resource instance
    Lists the messages in the user's mailbox.
    
    Args:
      userId: string, The user's email address. The special value `me` can be used to indicate the authenticated user. (required)
      maxResults: integer, Maximum number of messages to return. This field defaults to 100. The maximum allowed value for this field is 500.
      pageToken: string, Page token to retrieve a specific page of results in the list.
      q: string, Only return messages matching the specified query. Supports the same query format as the Gmail search box. For example, `"from:someuser@example.com rfc822msgid: is:unread"`. Parameter cannot be used when accessing the api using the gmail.metadata scope.
      includeSpamTrash: boolean, Include messages from `SPAM` and `TRASH` in the results.
      labelIds: string, Only return messages with labels that match all of the specifie

In [10]:
users.messages().list(userId='me').execute()

{'messages': [{'id': '180c0f5b1835a5af', 'threadId': '180c0f5b1835a5af'},
  {'id': '180c0a02db22e496', 'threadId': '180c0a02db22e496'},
  {'id': '180c09b43b06bf39', 'threadId': '180c09b43b06bf39'},
  {'id': '180c07fd1d2df18e', 'threadId': '180c07fd1d2df18e'},
  {'id': '180c066d4659c67f', 'threadId': '180c066d4659c67f'},
  {'id': '180c051a37ce800b', 'threadId': '180c051a37ce800b'},
  {'id': '180c038f8fcdc8f8', 'threadId': '180c038f8fcdc8f8'},
  {'id': '180bff3ec4d92a8a', 'threadId': '180bff3ec4d92a8a'},
  {'id': '180bf7e1111bc9ad', 'threadId': '180bf7e1111bc9ad'},
  {'id': '180bf4618edb3517', 'threadId': '180bf4618edb3517'},
  {'id': '180bf307c5d2d252', 'threadId': '180bf307c5d2d252'},
  {'id': '180bec7646ce8975', 'threadId': '180bebf5c71defcd'},
  {'id': '180bec1e50311067', 'threadId': '180bec1e50311067'},
  {'id': '180bebf5c71defcd', 'threadId': '180bebf5c71defcd'},
  {'id': '180beb36afe0b8ce', 'threadId': '180beb36afe0b8ce'},
  {'id': '180be9b43cc78282', 'threadId': '180be9b43cc78282

In [11]:
START_DATE = '2022/01/01'
END_DATE = '2022/01/10'

In [12]:
message_ids = []
next_page_token = None

while True:
    print(f'Processing in range between {START_DATE} and {END_DATE} using token {next_page_token}')
    if next_page_token:
        messages = users. \
            messages(). \
            list(
                userId='me', 
                q=f'after:{START_DATE} before:{END_DATE}',
                pageToken=next_page_token
            ). \
            execute()
        message_ids += messages['messages']
        next_page_token = messages.get('nextPageToken')
    else:
        messages = users. \
            messages(). \
            list(
                userId='me', 
                q=f'after:{START_DATE} before:{END_DATE}'
            ). \
            execute()
        message_ids = messages['messages']
        next_page_token = messages.get('nextPageToken')
    if next_page_token == None:
        break
        

Processing in range between 2022/01/01 and 2022/01/10 using token None
Processing in range between 2022/01/01 and 2022/01/10 using token 12950398023757421533
Processing in range between 2022/01/01 and 2022/01/10 using token 16588361506118913105
Processing in range between 2022/01/01 and 2022/01/10 using token 10524153043200532025
Processing in range between 2022/01/01 and 2022/01/10 using token 07452538926101471885


In [13]:
message_ids

[{'id': '17e413c017220429', 'threadId': '17e413c017220429'},
 {'id': '17e4113cd4050e8b', 'threadId': '17e4113cd4050e8b'},
 {'id': '17e40bb508e13dba', 'threadId': '17e40bb508e13dba'},
 {'id': '17e409d3e653f220', 'threadId': '17e409d3e653f220'},
 {'id': '17e403b4ab2219e8', 'threadId': '17e403b4ab2219e8'},
 {'id': '17e4025df4eeb048', 'threadId': '17e4025df4eeb048'},
 {'id': '17e400967f562a92', 'threadId': '17e400967f562a92'},
 {'id': '17e3ffc69e499538', 'threadId': '17e3ffc69e499538'},
 {'id': '17e3fefdc778147a', 'threadId': '17e3fefdc778147a'},
 {'id': '17e3fdf59d69e8fc', 'threadId': '17e3fdf59d69e8fc'},
 {'id': '17e3fb15978450b3', 'threadId': '17e3fb15978450b3'},
 {'id': '17e3f989ae7dc3be', 'threadId': '17e3f989ae7dc3be'},
 {'id': '17e3f77aed6dba8b', 'threadId': '17e3f77aed6dba8b'},
 {'id': '17e3f7172ad1ee39', 'threadId': '17e3f7172ad1ee39'},
 {'id': '17e3f5bd953e34a1', 'threadId': '17e3f5bd953e34a1'},
 {'id': '17e3f4b7645a402a', 'threadId': '17e3f4b7645a402a'},
 {'id': '17e3f134cbaae15

In [14]:
len(message_ids)

452